In [35]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


$\newcommand{\set}[2]{\big\{#1\,\ {\large:}\ \,#2\big\}}
\newcommand{\eps}{\varepsilon}
\newcommand{\abs}[1]{\left\vert#1\right\vert}
\newcommand{\ceil}[1]{\left\lceil#1\right\rceil}
\newcommand{\floor}[1]{\left\lfloor#1\right\rfloor}
\DeclareMathOperator{\sinal}{sinal}
$
# Os métodos da posição falsa e da secante

## $ \S 1 $ Descrição do método da posição falsa

Suponha que a função real contínua $ f $ troque de sinal dentro do intervalo $ [a, b] $. Assim como o método da bissecção, o **método da posição falsa** começa com as duas estimativas $ a $ e $ b $ para o zero. Mas em vez de tomar a próxima estimativa como o ponto médio destes, construímos a reta pelos pontos
$$
\big(a, f(a)\big) \quad \text{e} \quad \big(b,f(b)\big)
$$
e consideramos o ponto onde ela cruza o eixo-$x$, definindo $ \xi $ como sua primeira coordenada. Este seria exatamente o zero de $ f $ caso ela fosse linear, mas em geral isto não acontecerá. Portanto há três possibilidades:
* Se $ f(\xi) = 0 $, então $ \xi $ é um zero de $ f $ e podemos terminar.
* Se $ \sinal f(\xi) \neq \sinal f(a) $, então $ f $ troca de sinal em $ [a, \xi] $. Neste caso fazemos $ b \leftarrow \xi $.
* Se $ \sinal f(\xi) = \sinal f(a) $, então $ f $ troca de sinal em $ [\xi, b] $. Neste caso fazemos $ a \leftarrow \xi $.

Nos dois últimos casos repetimos o procedimento usando o novo intervalo $ [a, b] $ em lugar do original e assim sucessivamente, até que seja satisfeito o critério de parada. Observe que em cada iteração temos a garantia da existência de um zero dentro do intervalo sob consideração, pois $ f $ troca de sinal aí.


In [40]:
def imprime_solucao(xs, ys, freq=1):
    """
    Dados dois arrays xs e ys de mesmo comprimento, imprime um
    a cada 'freq' de seus valores por linha, na forma de uma
    tabela. Os 0-ésimos e últimos valores sempre são impressos.
    """
    def imprime_cabecalho():
        """
        Imprime o cabeçalho da tabela.
        """
        print("\n|       n      ", end="")
        print("    x_n            ", end="")
        print("    f(x_n)      |")
        print("|=================================================|")
        
        
    def imprime_linha(x, y, n):
        """
        Imprime uma das linhas da tabela.
        """
        if n == 0:
            print(f"|       a", end="")
        elif n == 1:
            print(f"|       b", end="")
        else:
            print(f"|      {n - 1:02}", end="")
        print(f"    {x:15.8f}", end="")
        print(f"    {y:15.8f}   |")
    
    
    N = len(xs)
    if freq == 0:          # Se freq == 0, imprime apenas os últimos valores.
        freq = N - 1
    imprime_cabecalho()
    for n in range(0, 2):
        imprime_linha(xs[n], ys[n], n)
    for n in range(2, N, freq):
        imprime_linha(xs[n], ys[n], n)
    if n != N - 1:
        imprime_linha(xs[N], ys[N])
    print("|_________________________________________________|\n")
        
    return None

In [41]:
def animador_posicao_falsa(f, a, b, N=4, titulo="", pausa=0.75):
    """
    Exibe uma animação do método da posição falsa aplicado a uma função.
    Entradas:
        * Uma função real contínua f.
        * As extremidades a < b do intervalo inicial.
        * O número de iterações a serem calculadas.
        * O título a ser exibido no topo do diagrama.
        * A pausa entre dois passos consecutivos da animação, em segundos.
    Saída:
        * Duas listas xs e ys contendo as estimativas para um zero
          e os valores de f nelas, respectivamente.
    Exibe:
        * Uma animação dos elementos relevantes numa janela pop-up.
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    
    def iteracao(a, b):
        """
        Aplica um passo do método da bissecção ao intervalo [a, b].
        Retorna o ponto médio e as extremidades do intervalo seguinte.
        """
        c = (a * f(b) - b * f(a)) / (f(b) - f(a)) 
        if np.sign(f(a)) != np.sign(f(c)):     # [a, c] contém um zero.
            return c, a, c
        else:                                  # [c, b] contém um zero.
            return c, c, b
        
    P = 200                                    # Número de pontos num plot.
    grossa = 2.0                               # Espessura grossa da traço.
    media = 1.75                               # Espessura média.
    fina = 1.0                                 # Espessura fina.
    dominio = np.linspace(a, b, P)             # Gera P nodos de a a b.
    xs = [a, b]                                # Armazenará as estimativas.
    # Criando as amostras para traçar as retas entre duas estimativas:
    xs_reta = [np.linspace(a, b, P)]
    ys_reta = [np.linspace(f(a), f(b), P)]
    for _ in range(N):                         # Preenchendo xs.
        c, a, b = iteracao(a, b)
        xs.append(c)
        xs_reta.append(np.linspace(a, b, P))   # Preenchendo xs_reta.
        ys_reta.append(np.linspace(f(a), f(b), P))    # Preenchendo ys_reta.
    ys = [f(x) for x in xs]                    # Armazena f das estimativas.
       
    # Listas contendo coordenadas x e y para plotagem das retas verticais:
    xs_vert = [np.linspace(xs[n], xs[n], P) for n in range(N + 2)]
    ys_vert = [np.linspace(0, ys[n], P) for n in range(N + 2)]                       

    # Desenhando o gráfico de f:
    plt.axhline(y=0.0, color='black', linestyle='-', linewidth=media)
    plt.xlabel('Eixo-$ x $')
    plt.ylabel('Eixo-$ y $')
    plt.title(titulo)
    plt.grid(True)
    plt.plot(dominio, f(dominio), label='$ y = f(x) $', linewidth=media)
    plt.legend()
    
    cmap = plt.get_cmap("tab10")
    # Marcando a no eixo-x e desenhando a reta vertical por ele:
    if pausa > 0:
        plt.pause(pausa)
    plt.plot(xs[0], 0, color='black', marker="|", mew=media)
    if pausa > 0:
            plt.pause(pausa)
    plt.plot(xs_vert[0], ys_vert[0], linestyle='-', linewidth=media, label='$ a $')
    plt.plot(xs[0], 0, color='black', marker="|", mew=media)
    plt.plot(xs[0], ys[0], color='black', marker="o", mew=.175)
    plt.legend()
    
    # Marcando b no eixo-x e desenhando a reta vertical por ele:
    if pausa > 0:
        plt.pause(pausa)
    plt.plot(xs[1], 0, color='black', marker="|", mew=media)
    if pausa > 0:
            plt.pause(pausa)
    plt.plot(xs_vert[1], ys_vert[1], linestyle='-', linewidth=media, label='$ b $')
    plt.plot(xs[1], 0, color='black', marker="|", mew=media)
    plt.plot(xs[1], ys[1], color='black', marker="o", mew=0.175)
    plt.legend()
    
    for n in range(2, N + 2):
        if pausa > 0:
            plt.pause(pausa)
        # Desenhando a reta entre as duas últimas estimativas:
        plt.plot(xs_reta[n - 2], ys_reta[n - 2], linestyle='--', linewidth=media)
        if pausa > 0:
            plt.pause(pausa)
        # Marcando x_n:
        plt.plot(xs[n], 0, color='black', marker="|", mew=media)
        if pausa > 0:
            plt.pause(pausa)
        # Desenhando a reta vertical por x_n e marcando (x_n, y_n):
        plt.plot(xs_vert[n], ys_vert[n], linestyle='-', linewidth=media,
                 label=f'$ x_{n - 1} $', color=cmap(n + 1))
        plt.plot(xs[n], 0, color='black', marker="|", mew=media)
        plt.plot(xs[n], ys[n], color='black', marker="o", mew=0.175)
    plt.legend()
        
    return xs, ys

## $ \S 2 $ Fórmulas para a próxima estimativa e para o erro

### $ 2.1 $ Fórmula para a próxima estimativa

A reta por dois pontos $ (x_0, y_0) $ e $ (x_1, y_1) $ é caracterizada pela igualdade da inclinação do segmento que liga $ (x_1, y_1) $ a $ (x_0, y_0) $ e a do segmento ligando um ponto $ (x, y) $ qualquer sobre a reta a $ (x_0, y_0) $. Em símbolos:
$$
\frac{y - y_0}{x - x_0} = \frac{y_1 - y_0}{x_1 - x_0}
$$
Equivalentemente, esta reta é descrita pela equação
$$
y = y_0 + \frac{y_1 - y_0}{x_1 - x_0}\,(x - x_0).
$$

No método da posição falsa utilizamos em cada passo a reta passando por $ \big(a,f(a)\big) $ e $ \big(b,f(b)\big) $, onde $ a $ e $ b $ são as extremidades do intervalo obtido no passo anterior. Substituindo estes valores acima, obtemos a equação
$$
y = f(a) + \frac{f(b) - f(a)}{b - a}\,(x - a).
$$
A próxima estimativa $ \xi $ para o zero é o único valor de $ x $ que faz esta expressão se anular:
$$
\boxed{\xi = \frac{af(b) - bf(a)}{f(b)-f(a)}}
$$ 

📝 Observe a simetria desta fórmula com respeito a $ a $ e $ b $ e o fato que o denominador é não-nulo pois $ f(a) $ e $ f(b) $ têm sinais opostos por hipótese.

### $ 2.2 $ Análise informal do erro

Suponha por concretude que o sinal de $ f(\xi) $ seja o mesmo que o de $ f(a) $, de modo que o intervalo seguinte seja $ [\xi, b] $. Podemos estimar o novo erro $ b - \xi $ em termos do anterior $ b - a $:
\begin{alignat*}{3}
b - \xi &= \frac{bf(b) - {bf(a)} -\big[af(b) - {bf(a)}\big]}{f(b) - f(a)} \\
& = \frac{f(b)}{f(b) - f(a)} (b - a)\,.
\end{alignat*}
Como por hipótese $ f(a) $ e $ f(b) $ têm sinais opostos, o fator que multiplica $ (b - a) $ está entre $ 0 $ e $ 1 $. Se $ f(b) $ for muito maior que $ f(a) $ em valor absoluto, este fator ficará próximo de $ 1 $. Se isto acontecer em todas as iterações, a convergência ao zero será lenta; esta situação é ilustrada pela animação abaixo.

In [42]:
a = 0          # Extremidade esquerda do intervalo inicial, onde f vale -3.
b = 4          # Extremidade direita, onde f vale 57.
N = 8          # Número de iterações desejado.
pausa = 0.5    # Intervalo de tempo entre cada passo da animação, em segundos.
f = lambda x: x**3 - x - 3    # Função à qual o método será aplicado.
# Título a ser exibido no topo do diagrama:
titulo = ("Método da posição falsa para $ y = x^3 - x - 3,\ a = 0,\ b = 4 $."
          "\nAproximação lenta por um único lado.")

xs, ys = animador_posicao_falsa(f, a, b, N, titulo, pausa)
imprime_solucao(xs, ys)


|       n          x_n                f(x_n)      |
|=================================================|
|       a         0.00000000        -3.00000000   |
|       b         4.00000000        57.00000000   |
|      01         0.20000000        -3.19200000   |
|      02         0.40151515        -3.33678512   |
|      03         0.60052096        -3.38395783   |
|      04         0.79103006        -3.29605996   |
|      05         0.96644712        -3.06376615   |
|      06         1.12118428        -2.71179488   |
|      07         1.25192491        -2.28976299   |
|      08         1.35805522        -1.85337501   |
|_________________________________________________|



![Método da posição falsa](fig_2-4_posicao_falsa_aprox_lenta.png "Title")

Não é possível calcular precisamente e de maneira geral o erro cometido pelo método da posição falsa. Porém na situação descrita acima, seu desempenho é consideravelmente pior que o do método do bissecção. Por este motivo ele raramente é empregado na prática.

## $ \S 3 $ Implementação do método da posição falsa

Como o método da bissecção e da posição falsa só diferem na fórmula usada para obter a nova estimativa para o zero, as duas implementações são muito parecidas.

In [2]:
def posicao_falsa(f, a, b, eps, max_iter):
    """
    Utiliza o método da posição falsa para localizar um zero de uma função.
    Entradas:
        * A função real contínua f.
        * As extremidades a < b de um intervalo onde f troca de sinal.
        * A distância máxima tolerada eps da estimativa a um zero.
        * O número máximo max_iter de iterações.
    Saída:
        * A lista xs das estimativas e a lista ys dos valores de f nelas.
    Imprime:
        * A última estimativa, o valor de f aí, o número de iterações
          e uma cota para o erro.
    """
    from numpy import sign
    
    
    iteracoes = 0                         # Contador do número de iterações.
    f_a = f(a)                            # Gravando o valor de f em a.
    f_b = f(b)                            # Gravando o valor de f em b.
    xs = [a, b]                           # Lista que armazenará as estimativas.
    ys = [f_a, f_b]                       # Armazenará os valores de f nos x.
    if eps <= 0:                          # Valor inválido de eps.
        raise ValueError("A tolerância deve ser positiva!")
    if f_a == 0:                          # a é um zero.
        print("a é um zero exato.")
        return a
    elif f_b == 0:                        # b é um zero.
        print("b é um zero exato.")
        return b
    elif sign(f_a) == sign(f_b):          # Erro: [a, b] não contém um zero.
        raise ValueError("A função assume valores de mesmo"
                         "sinal nas extremidades dadas!")
    
    while (b - a) >= eps and iteracoes <= max_iter:
        # Próxima estimativa:
        x = (a * f(b) - b * f(a)) / (f(b) - f(a)) 
        f_x = f(x)                        # Gravando o valor de f em x.
        xs.append(x)
        ys.append(f_x)
        if f_x == 0:
            print("Encontrado um zero exato.")
            return x
        elif sign(f_a) != sign(f_x):      # O intervalo [a, x] contém um zero.
            b = x                         # Tome o novo b como sendo x.
        else:                             # O intervalo [x, b] contém um zero.
            a = x                         # Tome o novo a como sendo x.
            f_a = f_x
        iteracoes += 1
    
    print(f"Encontrado um zero aproximado:\n{x:15.9f}")
    print(f"após {iteracoes} iterações, com erro de no máximo {b - a}.")
    print(f"O valor da função neste ponto é:\n{f(x):15.9f}")
    
    return xs, ys

## $ \S 4 $ Problemas

**Problema 1:** Usando o método da posição falsa com precisão de três dígitos decimais:

(a) Encontre uma raiz positiva da equação $ \sin x = \frac{x}{2} $.

(a) Encontre a primeira raiz positiva da equação $ \sin x = \frac{x}{n} $ para $ n = 2, 3, \dots, 20 $.

*Solução:*

**Problema 2:** Calcule com ajuda do computador, mas sem usar a implementação acima, as três primeiras iterações do método da posição falsa para estimar uma raiz das equações abaixo nos intervalos indicados. Esboce também os gráficos das funções utilizadas:

(a) $ \tan x - \frac{1}{1 + x^2} = 0 $, $ 0 \le x \le \frac{\pi}{2} $.

(b) $ x^2 = 2 $, $ 0 \le x \le 2 $.

(c) $ x \ln x = 1 $, $ 1 \le x \le 2 $.

(d) $ \cos x = x $, $ 0 \le x \le \frac{\pi}{2} $.

*Solução:*

**Problema 3:** Vimos no caderno anterior que no método da bissecção os comprimentos do intervalo anterior e do atual estão relacionados pela fórmula 
$$
\abs{I_n} = c \abs{I_{n-1}} \quad \text{com} \quad c = \frac{1}{2}
$$ 

(a) Estime a constante $ c $ que relaciona as duas para o método da posição falsa aplicado ao problema de se encontrar a raiz positiva da equação
$$
x^{8} - 1 = 0
$$
usando como intervalo inicial $ [a, b] = [0, 2] $.

(b) Conclua se seria melhor usar o método da bissecção ou o método da posição falsa neste caso. 